In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range
import random

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 100000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789-+ '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char
#ctable.char_indices
#ctable.chars

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
countAdd = 0
countSub = 0
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b),reverse=True))  
    if key in seen:
        continue
    seen.add(key)
    ranAddSub = random.randint(0,1)
    #print (ranAddSub)
    if ranAddSub==0:
        countAdd+=1
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
        ans += ' ' * (DIGITS + 1 - len(ans))
    else :
        countSub+=1
        a,b = sorted((a, b),reverse=True)
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a - b)
        ans += ' ' * (DIGITS + 1 - len(ans))
    #print(query)
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))
print(countAdd , countSub)

Generating data...
Total addition questions: 100000
50096 49904


In [8]:
print(questions[:5], expected[:5])

['314-6  ', '62-3   ', '6+15   ', '756+613', '5+80   '] ['308 ', '59  ', '21  ', '1369', '85  ']


# Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:80000]
train_y = y[:80000]
test_x = x[80000:]
test_y = y[80000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(72000, 7, 13)
(72000, 4, 13)
Validation Data:
(8000, 7, 13)
(8000, 4, 13)
Testing Data:
(20000, 7, 13)
(20000, 4, 13)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False  True False False
   False False]
  [False False False False False False False False False False False
   False  True]
  [False False False False False  True False False False False False
   False False]
  [False  True False False False False False False False False False
   False False]
  [False False False False False False False  True False False False
   False False]
  [False False False False False False False False  True False False
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False False False False False False False
   False  True]
  [False False False False False  True False False False False False
   False False]
  [False False False False False False False  True False False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False  True Fals

# Build Model

In [12]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 13)             0         
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [13]:
for iteration in range(50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 12s 171us/step - loss: 1.8428 - acc: 0.3280 - val_loss: 1.7541 - val_acc: 0.3443
Q 830-0   T 830  ☒ 37  
Q 239-12  T 227  ☒ 317 
Q 852-751 T 101  ☒ 10  
Q 537+67  T 604  ☒ 100 
Q 924-52  T 872  ☒ 337 
Q 803-6   T 797  ☒ 37  
Q 943+905 T 1848 ☒ 1100
Q 505+66  T 571  ☒ 100 
Q 445-89  T 356  ☒ 33  
Q 890-15  T 875  ☒ 600 

--------------------------------------------------
Iteration 1
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 10s 142us/step - loss: 1.6777 - acc: 0.3754 - val_loss: 1.5990 - val_acc: 0.4056
Q 995-307 T 688  ☒ 701 
Q 68+889  T 957  ☒ 901 
Q 509-210 T 299  ☒ 255 
Q 574-371 T 203  ☒ 35  
Q 759-513 T 246  ☒ 355 
Q 156-90  T 

72000/72000 [==============================] - 10s 143us/step - loss: 0.4222 - acc: 0.8576 - val_loss: 0.3866 - val_acc: 0.8721
Q 809+30  T 839  ☑ 839 
Q 66+430  T 496  ☒ 596 
Q 25+476  T 501  ☑ 501 
Q 599-571 T 28   ☒ 18  
Q 625-53  T 572  ☑ 572 
Q 653-99  T 554  ☑ 554 
Q 303+285 T 588  ☑ 588 
Q 88+697  T 785  ☑ 785 
Q 94+985  T 1079 ☒ 1089
Q 676-398 T 278  ☒ 277 

--------------------------------------------------
Iteration 14
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 10s 143us/step - loss: 0.3382 - acc: 0.8952 - val_loss: 0.3259 - val_acc: 0.8925
Q 734-93  T 641  ☑ 641 
Q 8+348   T 356  ☑ 356 
Q 15+408  T 423  ☒ 434 
Q 658+34  T 692  ☒ 792 
Q 582+997 T 1579 ☒ 1570
Q 42+215  T 257  ☒ 258 
Q 508+871 T 1379 ☑ 1379
Q 604-408 T 196  ☒ 107 
Q 859+8   T 867  ☒ 877 
Q 324-123 T 201  ☑ 201 

--------------------------------------------------
Iteration 15
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [===

72000/72000 [==============================] - 10s 143us/step - loss: 0.0805 - acc: 0.9754 - val_loss: 0.1653 - val_acc: 0.9426
Q 631-3   T 628  ☑ 628 
Q 680-192 T 488  ☑ 488 
Q 863+158 T 1021 ☑ 1021
Q 933-3   T 930  ☑ 930 
Q 31+409  T 440  ☒ 540 
Q 497+307 T 804  ☒ 704 
Q 792+22  T 814  ☑ 814 
Q 878-755 T 123  ☑ 123 
Q 51+950  T 1001 ☒ 100 
Q 987+69  T 1056 ☒ 1046

--------------------------------------------------
Iteration 28
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 10s 142us/step - loss: 0.0603 - acc: 0.9833 - val_loss: 0.0831 - val_acc: 0.9729
Q 290+984 T 1274 ☑ 1274
Q 395+59  T 454  ☑ 454 
Q 921-352 T 569  ☑ 569 
Q 736+89  T 825  ☑ 825 
Q 640-22  T 618  ☑ 618 
Q 761+66  T 827  ☑ 827 
Q 911+50  T 961  ☒ 971 
Q 144-47  T 97   ☑ 97  
Q 87+134  T 221  ☑ 221 
Q 18+800  T 818  ☑ 818 

--------------------------------------------------
Iteration 29
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [===

72000/72000 [==============================] - 11s 147us/step - loss: 0.0354 - acc: 0.9900 - val_loss: 0.0586 - val_acc: 0.9807
Q 288-54  T 234  ☑ 234 
Q 488-73  T 415  ☑ 415 
Q 411-28  T 383  ☑ 383 
Q 854-287 T 567  ☑ 567 
Q 36+920  T 956  ☑ 956 
Q 917-593 T 324  ☑ 324 
Q 4+629   T 633  ☑ 633 
Q 464+646 T 1110 ☑ 1110
Q 591-32  T 559  ☑ 559 
Q 3+542   T 545  ☑ 545 

--------------------------------------------------
Iteration 42
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 10s 142us/step - loss: 0.0454 - acc: 0.9865 - val_loss: 0.0821 - val_acc: 0.9715
Q 58+417  T 475  ☑ 475 
Q 357+184 T 541  ☑ 541 
Q 490+46  T 536  ☑ 536 
Q 551-172 T 379  ☑ 379 
Q 444-69  T 375  ☑ 375 
Q 450+39  T 489  ☑ 489 
Q 34+792  T 826  ☑ 826 
Q 500-69  T 431  ☑ 431 
Q 527+678 T 1205 ☑ 1205
Q 24+13   T 37   ☑ 37  

--------------------------------------------------
Iteration 43
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [===

# Testing

In [14]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
equalAnswer = 0
predictions = model.predict_classes(test_x)

for i in range(len(test_x)):
    predAnswer = ctable.decode(predictions[i],False)
    correctAnswer = ctable.decode(test_y[i])
    if(predAnswer == correctAnswer):
        equalAnswer+=1        

accuracyAnswer = equalAnswer/len(test_y)
accuracyAnswer

MSG : Prediction


0.96885